### Naive Bayes Algoritması ile E-mail Spam Kontrolü

In [1]:
import pandas as pd

#### Veri Ön İşleme

In [2]:
# Stop Word 
# Stem işlemi vb

In [10]:
df = pd.read_csv("spam_ham_dataset.csv")
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [11]:
df = df[['label_num', 'text']]
df.head()

,label_num,text
0,0,Subject: enron methanol ; meter # : 988291\r\n...
1,0,"Subject: hpl nom for january 9 , 2001\r\n( see..."
2,0,"Subject: neon retreat\r\nho ho ho , we ' re ar..."
3,1,"Subject: photoshop , windows , office . cheap ..."
4,0,Subject: re : indian springs\r\nthis deal is t...


In [29]:
!pip install nltk

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------- ----- 1.3/1.5 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 13.2 MB/s eta 0:00:00


In [52]:
# Running,
# running
# runNing
# RUNNING!
# run

# RegularExpression
import re
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#from nltk.stem import WordNetLemmatizer

# stem işlemi -> kelimelerin köküne indirme. -> Running -> Run
# stopword -> the, is, at
nltk.download('stopwords')
#nltk.download('wordnet')

ps = PorterStemmer() # SnowballStemmer, LancasterStemmer
#ps = WordNetLemmatizer()
def clean_text(text):
    text = text.lower() # tüm harfleri küçük hale getirdim
    
    #text = re.sub(r'\d+', '', text) # metindeki tüm numerik değerleri kaldırdım
    
    text = text.translate(str.maketrans('','', string.punctuation)) # metindeki bütün noktalama işaretlerini kaldırır.
    
    words = text.split() # Merhaba Dünya => [Merhaba,Dünya]
    
    words = [ word for word in words if not word.isdigit() ] # Kelimelerin yalnızca numerik olanlarını ele.

    words = [ ps.stem(word) for word in words if word not in stopwords.words('english')  ] # kelimeleri köküne indirip, stopwordleri (the,is,at vb) kaldırdık.

    return ' '.join(words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PC1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [53]:
# Subject: hpl nom for january 9 , 2001\r\n( see
# subject hpl nom for january see
clean_text("The Subject: hpl nom for january january running 9 , 2001 ( see ) p2p 4.5g 3d january")

'subject hpl nom januari januari run see p2p 45g 3d januari'

In [49]:
# Win $$$ NOW !!!! -> win now $
# Win $$ NOW!!! -> win now $
df.head()

,label_num,text
0,0,Subject: enron methanol ; meter # : 988291\r\n...
1,0,"Subject: hpl nom for january 9 , 2001\r\n( see..."
2,0,"Subject: neon retreat\r\nho ho ho , we ' re ar..."
3,1,"Subject: photoshop , windows , office . cheap ..."
4,0,Subject: re : indian springs\r\nthis deal is t...


In [54]:
df['clean_text'] = df['text'].apply(clean_text)
df.head()

,label_num,text,clean_text
0,0,Subject: enron methanol ; meter # : 988291\r\n...,subject enron methanol meter follow note gave ...
1,0,"Subject: hpl nom for january 9 , 2001\r\n( see...",subject hpl nom januari see attach file hplnol...
2,0,"Subject: neon retreat\r\nho ho ho , we ' re ar...",subject neon retreat ho ho ho around wonder ti...
3,1,"Subject: photoshop , windows , office . cheap ...",subject photoshop window offic cheap main tren...
4,0,Subject: re : indian springs\r\nthis deal is t...,subject indian spring deal book teco pvr reven...


In [58]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X = df["clean_text"]
y = df["label_num"]

# tf-idf -> metinleri sayısal vektörlere dönüştür.

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df["clean_text"]) #Makine öğrenmesi için daha doğru bir veri.

# [ 
#  [ 0.8, 0.5, 0.3, 0 ,0 ,0],
#  [ 0.4, 1, 0.2, 0 , 0]
# ]


X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [59]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [61]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95       742
           1       1.00      0.74      0.85       293

    accuracy                           0.93      1035
   macro avg       0.95      0.87      0.90      1035
weighted avg       0.93      0.93      0.92      1035

